In [1]:
import re
from dateparser.search import search_dates
import os
import pandas as pd
import import_ipynb
from Tesseract import *

importing Jupyter notebook from Tesseract.ipynb


In [18]:
# DataDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
# os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")

In [3]:
def get_invoice_nums(all_words):
    inv_nums = []
    invoice_no_re = r'^[0-9a-zA-Z-:/]+$'
    for word in all_words:
        if not re.search('\d', word['text']):
            continue
        result = re.findall(invoice_no_re,word['text'])
        if result:
            inv_nums.append({
                'text': word['text'],
                'x1': word['left'],
                'y1': word['top'],
                'x2': word['left'] + word['width'],
                'y2': word['top'] + word['height']
            })

    return inv_nums

In [4]:
def checkAmt(date):
    date = date.lower()
    date = date.strip()
    validation = {
        "isNum" : 0,
        "isWord" : 0,
    }
    numChecker = 1
    wordChecker = 1
        
    for i in range(len(date)):
        char = date[i]
        
        if ord(char)>=48 and ord(char)<=57:
            pass
        else:
            numChecker=0

    for i in range(len(date)):
        char = date[i]
        
        if ord(char)>=97 and ord(char)<=122:
            pass
        else:
            wordChecker=0
        
    validation["isWord"] = wordChecker
    validation["isNum"] = numChecker

    if 1 in validation.values():
        return False
    else:
        return True

In [5]:
def isNum(text):
    numChecker = 1
    for i in range(len(text)):
        char = text[i]
        
        if ord(char)>=48 and ord(char)<=57:
            pass
        else:
            numChecker=0
            break
    if numChecker==1:
        return True
    else:
        return False

In [6]:
def get_amounts(all_words):
    amounts = []
    amount_re = r"^\$?\d+(,\d{3})*(\.\d+)?$"
    for word in all_words:
        if not re.search(amount_re, word['text']):
            continue
        if len(word)<3:
            continue
        result = re.findall(amount_re,word['text'])
        if result:
            if not checkAmt(word["text"]):
                continue
            if word["text"][0]=="0" or word["text"][0]==0:
                continue
            if isNum(word["text"]) and float(word["text"])>100:
                amounts.append({
                'text': word['text'],
                'x1': word['left'],
                'y1': word['top'],
                'x2': word['left'] + word['width'],
                'y2': word['top'] + word['height']
                })
            if "," in word["text"]:
                amounts.append({
                'text': word['text'],
                'x1': word['left'],
                'y1': word['top'],
                'x2': word['left'] + word['width'],
                'y2': word['top'] + word['height']
                })

    return amounts

In [7]:
def check(date):
    date = date.lower()
    date = date.strip()
    validation = {
        "isNum" : 0,
        "isWord" : 0,
        "isOther" : 0,
    }
    numChecker = 1
    wordChecker = 1
    
    for i in range(len(date)):
        char = date[i]
        if char in ["@", "&", "%", "$", "!", "#", "*"]:
            validation["isOther"]=1
        
    for i in range(len(date)):
        char = date[i]
        
        if ord(char)>=48 and ord(char)<=57:
            pass
        else:
            numChecker=0

    for i in range(len(date)):
        char = date[i]
        
        if ord(char)>=97 and ord(char)<=122:
            pass
        else:
            wordChecker=0
        
    validation["isWord"] = wordChecker
    validation["isNum"] = numChecker

    if 1 in validation.values():
        return False
    else:
        return True
    
        


In [8]:
def get_dates(all_text, all_words):
    dates, all_dates = [], []
    indices = []
    matches = search_dates(all_text)

    
    for match in matches:
        text = match[0]
        text = text.strip()

        for i, element in enumerate(all_words):
            if element["text"]!=text:
                pass
            elif element["text"]==text:
                date = ''
                left, top, right, bottom = [], [], [], []
                date += ' ' + all_words[i]['text']
                left.append(all_words[i]['left'])
                top.append(all_words[i]['top'])
                right.append(all_words[i]['left'] + all_words[i]['width'])
                bottom.append(all_words[i]['top'] + all_words[i]['height'])
                date = date.strip()
                if check(date):
                    all_dates.append({
                        'text': date.strip(),
                        'x1': min(left),
                        'y1': min(top),
                        'x2': max(right),
                        'y2': max(bottom)
                    })
            elif element["text"]==text.split(" ")[0]:
                date = '' 
                left, top, right, bottom = [], [], [], []
                j = i
                for a in range(3):
                    date += ' ' + all_words[j]['text']
                    left.append(all_words[j]['left'])
                    top.append(all_words[j]['top'])
                    right.append(all_words[j]['left'] + all_words[j]['width'])
                    bottom.append(all_words[j]['top'] + all_words[j]['height'])
                    j+=1
                date = date.strip()
                if check(date):
                    all_dates.append({
                        'text': date.strip(),
                        'x1': min(left),
                        'y1': min(top),
                        'x2': max(right),
                        'y2': max(bottom)
                    })
    
    return all_dates


In [9]:
def get_candidates(file):
        image = cv2.imread(file)
        data = ocr(file)
        data["left"]/=image.shape[1]
        data["top"]/=image.shape[0]
        data["width"]/=image.shape[1]
        data["height"]/=image.shape[0]
        data["X"]/=image.shape[1]
        data["Y"]/=image.shape[0]

        all_words = []
        for idx in data.index:
            word = data["text"][idx]
            if word.strip() != "":
                all_words.append({
                    'text': data['text'][idx],
                    'left': data['left'][idx],
                    'top': data['top'][idx],
                    'width': data['width'][idx],
                    'height': data['height'][idx]})
        text = ' '.join([word['text'].strip() for word in all_words])

        invoice_date_candidates = get_dates(text, all_words)
    
        total_amount_candidates = get_amounts(all_words)
    
        invoice_no_candidates = get_invoice_nums(all_words)

        candidate_data = {
            'invoice_no': invoice_no_candidates,
            'invoice_date': invoice_date_candidates,
            'total_amount': total_amount_candidates
        }
        Dataframes = []
        for key in candidate_data.keys():
            df = pd.DataFrame(candidate_data[key])
            df["Class"] = key
            Dataframes.append(df)
        return data, Dataframes

In [16]:
# D, DF = get_candidates("20.jpeg")

In [17]:
# DF[0]